5. Multi-output: 개별적으로 MLP 출력 모형을 만들어 각 출력 시계열을 처리함

In [8]:
from tensorflow.python.client import device_lib
import keras.backend.tensorflow_backend as K
#print(device_lib.list_local_devices())
from numpy import array 
from numpy import array 
from numpy import hstack 
from keras.models import Model 
from keras.layers import Input 
from keras.layers import Dense

In [2]:
# split a multivariate sequence into samples 
def split_sequences_multi(sequences, n_steps): 
    X, y = list(), list() 
    for i in range(len(sequences)): 
        # find the end of this pattern 
        end_ix = i + n_steps 
        # check if we are beyond the dataset 
        if end_ix > len(sequences)-1: 
            break 
        # gather input and output parts of the pattern 
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :] # 이 부분이 다름
        X.append(seq_x) 
        y.append(seq_y) 
    return array(X), array(y)

In [3]:
# 예제 데이터 셋 정의
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [4]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences_multi(dataset, n_steps)

In [5]:
# flatten input 
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

In [6]:
# separate output 
y1 = y[:, 0].reshape((y.shape[0], 1)) 
y2 = y[:, 1].reshape((y.shape[0], 1)) 
y3 = y[:, 2].reshape((y.shape[0], 1))

In [9]:
with K.tf.device('/gpu:0'): #'/cpu:0'
    # define model 
    visible = Input(shape=(n_input,)) 
    dense = Dense(100, activation='relu')(visible)
    # define output 1 
    output1 = Dense(1)(dense) 
    # define output 2 
    output2 = Dense(1)(dense) 
    # define output 3
    output3 = Dense(1)(dense)
    # tie together 
    model = Model(inputs=visible, outputs=[output1, output2, output3])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [10]:
# 예측
x_input = array([[70,75,145], [80,85,165], [90,95,185]]) 
x_input = x_input.reshape((1, n_input)) 
yhat = model.predict(x_input, verbose=0)
print(yhat)

[array([[100.016205]], dtype=float32), array([[104.99819]], dtype=float32), array([[205.03056]], dtype=float32)]
